# ValidMind for model validation 3 — Developing potential challenger models

Learn how to use ValidMind for your end-to-end model validation process with our series of four introductory notebooks. In this third notebook, develop potential challenger models and then pass your models and their predictions to ValidMind.

A *challenger model* is an alternate model that attempt to outperform the champion model, ensuring that the best performing fit-for-purpose model is always considered for deployment. Challenger models also help avoid over-reliance on a single model, and allow testing of new features, algorithms, or data sources without disrupting the production lifecycle.

## Prerequisites

In order to develop potential challenger models with this notebook, you'll need to first have:

- [x] Registered a model within the ValidMind Platform and granted yourself access to the model as a validator
- [x] Installed the ValidMind Library in your local environment, allowing you to access all its features
- [x] Learned how to import and initialize datasets for use with ValidMind
- [x] Understood the basics of how to run and log tests with ValidMind

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Need help with the above steps?</b></span>
<br></br>
Refer to the first two notebooks in this series:

- <a href="1-set_up_validmind_for_validation.ipynb" style="color: #DE257E;"><b>1 — Set up the ValidMind Library for validation</b></a>
- <a href="2-start_validation_process.ipynb" style="color: #DE257E;"><b>2 — Start the model validation process</b></a>

</div>



## Setting up

This section should be quite familiar to you — as we performed the same actions in the previous notebook, **[2 — Start the model validation process](2-start_validation_process.ipynb)**.

### Initialize the ValidMind Library

As usual, let's first connect up the ValidMind Library to our model we previously registered in the ValidMind Platform:

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Inventory** and select the model you registered for this "ValidMind for model validation" series of notebooks.

3. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, [load your model identifier credentials from an `.env` file](https://docs.validmind.ai/developer/model-documentation/store-credentials-in-env-file.html) or replace the placeholder with your own code snippet:

In [ ]:
# Make sure the ValidMind Library is installed

%pip install -q validmind

# Load your model identifier credentials from an `.env` file

%load_ext dotenv
%dotenv .env

# Or replace with your code snippet

import validmind as vm

vm.init(
    # api_host="...",
    # api_key="...",
    # api_secret="...",
    # model="...",
)

### Import the champion model

Next, we'll import the champion model submitted by the model development team as we used in the last notebooks (**[lr_model_champion.pkl](lr_model_champion.pkl)**) and load in the sample [Bank Customer Churn Prediction](https://www.kaggle.com/datasets/shantanudhakadd/bank-customer-churn-prediction) dataset used to develop the model that we will independently preprocess:

In [ ]:
# Import the champion model
import pickle

with open("lr_model_champion.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
# Load the sample dataset
from validmind.datasets.classification import customer_churn as demo_dataset

print(
    f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}' \n\t• Class labels: {demo_dataset.class_labels}"
)

raw_df = demo_dataset.load_data()
raw_df.head()

In [ ]:
# Initialize the ValidMind dataset object
vm_raw_dataset = vm.init_dataset(
    dataset=raw_df,
    input_id="raw_dataset",
    target_column="Exited",
)

### Preprocess the dataset

We’ll apply a simple rebalancing technique to the dataset before continuing:

In [ ]:
import pandas as pd

raw_copy_df = raw_df.sample(frac=1)  # Create a copy of the raw dataset

# Create a balanced dataset with the same number of exited and not exited customers
exited_df = raw_copy_df.loc[raw_copy_df["Exited"] == 1]
not_exited_df = raw_copy_df.loc[raw_copy_df["Exited"] == 0].sample(n=exited_df.shape[0])

balanced_raw_df = pd.concat([exited_df, not_exited_df])
balanced_raw_df = balanced_raw_df.sample(frac=1, random_state=42)

Let’s also quickly remove highly correlated features from the dataset using the output from a ValidMind test.

As you know, before we can run tests you’ll need to initialize a ValidMind dataset object:

In [ ]:
# Register new data and now 'balanced_raw_dataset' is the new dataset object of interest
vm_balanced_raw_dataset = vm.init_dataset(
    dataset=balanced_raw_df,
    input_id="balanced_raw_dataset",
    target_column="Exited",
)

With our balanced dataset initialized, we can then run our test and utilize the output to help us identify the features we want to remove:

In [ ]:
# Run HighPearsonCorrelation test with our balanced dataset as input and return a result object
corr_result = vm.tests.run_test(
    test_id="validmind.data_validation.HighPearsonCorrelation",
    params={"max_threshold": 0.3},
    inputs={"dataset": vm_balanced_raw_dataset},
)

In [ ]:
# From result object, extract table from `corr_result.tables`
features_df = corr_result.tables[0].data
features_df

In [ ]:
# Extract list of features that failed the test
high_correlation_features = features_df[features_df["Pass/Fail"] == "Fail"]["Columns"].tolist()
high_correlation_features

In [ ]:
# Extract feature names from the list of strings
high_correlation_features = [feature.split(",")[0].strip("()") for feature in high_correlation_features]
high_correlation_features

We can then re-initialize the dataset with a different input_id and the highly correlated features removed and re-run the test for confirmation:

In [ ]:
# Remove the highly correlated features from the dataset
balanced_raw_no_age_df = balanced_raw_df.drop(columns=high_correlation_features)

# Re-initialize the dataset object
vm_raw_dataset_preprocessed = vm.init_dataset(
    dataset=balanced_raw_no_age_df,
    input_id="raw_dataset_preprocessed",
    target_column="Exited",
)

In [ ]:
# Re-run the test with the reduced feature set
corr_result = vm.tests.run_test(
    test_id="validmind.data_validation.HighPearsonCorrelation",
    params={"max_threshold": 0.3},
    inputs={"dataset": vm_raw_dataset_preprocessed},
)

## Split the preprocessed dataset

With our dummy model imported, raw dataset rebalanced with highly correlated features removed, let's now **spilt our dataset into train and test** in preparation for training potential challenger models.

To start, let's grab the first few rows from the `balanced_raw_no_age_df` dataset we initialized earlier:

In [ ]:
balanced_raw_no_age_df.head()

Before training the models, we need to encode the categorical features in the dataset:

- Use the `OneHotEncoder` class from the `sklearn.preprocessing` module to encode the categorical features.
- The categorical features in the dataset are `Geography` and `Gender`.

In [ ]:
balanced_raw_no_age_df = pd.get_dummies(
    balanced_raw_no_age_df, columns=["Geography", "Gender"], drop_first=True
)
balanced_raw_no_age_df.head()

Splitting our dataset into training and testing is essential for proper validation testing, as this helps assess how well the model generalizes to unseen data:

- We begin by dividing our preprocessed dataset into feature variables (`X`) and the target variable (`y`), which indicates whether a customer exited.
- Using `train_test_split`, we randomly allocate 80% of the data to the training set (`X_train`, `y_train`) and 20% to the test set (`X_test`, `y_test`), ensuring both sets are representative of the full distribution.

In [ ]:
# Split the input and target variables
X = balanced_raw_no_age_df.drop("Exited", axis=1)
y = balanced_raw_no_age_df["Exited"]
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)